In [ ]:
import os
import faiss
import json
import glob

from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.docstore import InMemoryDocstore
from langchain.schema import Document
from uuid import uuid4
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough


In [ ]:
os.environ["OpenAI_API_KEY"] = os.getenv("GPT_API")

model = ChatOpenAI(model ="gpt-4o")

In [ ]:
path = "food_recipe/"

merge_recipes = []

for path in glob.glob(os.path.join(path,"*json")):
    with open(path, 'r', encoding="utf-8") as file:
        data = json.load(file)
        merge_recipes.append(data)

In [ ]:
document_recipe = [
    Document(page_content=str(recipe), metadata={"요리명": recipe["요리명"],"요리재료" : recipe["요리재료"], "기본정보" : recipe["기본정보"], "조리순서" : recipe["조리순서"]})
    for recipe in data
]

In [ ]:
embeddings = OpenAIEmbeddings(model = "text-embedding-3-small")

In [ ]:
index  = faiss.IndexFlatL2(len(embeddings.embed_query("레시피")))

recipe_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

In [ ]:
uuids = [str(uuid4()) for _ in range(len(document_recipe))]

recipe_store.add_documents(documents=document_recipe, ids = uuids)

In [ ]:
recipe_store.save_local("./food_db")
recipes = FAISS.load_local("./food_db." ,embeddings, allow_dangerous_deserialization=True)

In [ ]:
retriever = recipes.as_retriever(search_type="similarity", search_kwargs={"k": 1})

In [ ]:
retriever.get_relevant_documents("미역국 레시피")

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 주어진 데이터로만 답변을 할 수 있어."),
    ("system", "너는 요리의 전문가야."),
    
    ("system", "질문에 답변할 때는 항상 데이터를 세밀히 학습하고 정확한 답변을 생성해야 해."),
    ("system", "만약 데이터 안에 유저의 질문이 없다면, 양해를 구하고 관련 데이터를 제공할 수 없음을 알리도록 해."),
    ("system", "질문에 계산이 필요한 경우, 데이터를 기반으로 정확하게 계산하여 결과를 제공해야 합니다."),
    ("system", "질문에 대한 답변을 생성 하기전에 검증을 마친 후에 생성해줘."),
    ("system", "부족한 부분이 있다고 생각하면, 다른 데이터를 참조해줘"),

    
    ("system", "다음은 답변 형식의 예시야: \n"
               "user: 미역국을 만들고 싶어.\n"
               "ai: 미역국의 재료는 ~입니다.\n"
               "ai: 미역국을 만드는 순서는 다음과 같습니다:\n"
               "ai: 첫 번째 ~~~, 두 번째 ~~~입니다.\n"),
    
    ("system", "또 다른 예시를 들어줄게:\n"
               "user: 내가 만들려고 하는 미역국의 칼로리는 얼마야?\n"
               "ai: 미역국의 칼로리는 1인분당 ~kcal입니다. 이 레시피는 ~인분 기준이므로, 총 칼로리는 ~kcal입니다.\n"),
    
    ("system", "추가 예시:\n"
               "user: 5인분의 레시피로 수정해줘.\n"
               "ai: 현재 미역국 레시피는 ~인분 기준입니다. 5인분으로 수정된 재료는 다음과 같습니다:\n"
               "ai: ~~~.\n"
               "이때도 순서를 전부 알려줘"),
    
    ("user", "다음과 같은 데이터를 학습해:\n{data}"),
    ("user", "그리고 질문에 답해:\n{question}")
])

In [ ]:
contextual_prompt = prompt

In [ ]:
class DebugPassThrough(RunnablePassthrough):
    
    def invoke(self, *args, **kwargs):
        output = super().invoke(*args,**kwargs)
        return output
    
class ContextToText(RunnablePassthrough):
    def invoke(self, inputs, config = None, **kwargs):
        return {"data": inputs["data"], "question": inputs["question"]}
    
rag_chain_debug = {
    "data" : retriever,
    "question" : DebugPassThrough()
    
} | DebugPassThrough() | ContextToText()| contextual_prompt | model

In [ ]:
while True:
    print("-----------------------------")
    
    query = input("질문을 입력해 주세요 (break 입력시 종료됩니다) : ")
    
    if query.lower() == "break":
        break
    
    response = rag_chain_debug.invoke(query)
    
    print("Question : ", query)
    print(response.content)